# Clave pública

In [46]:
raw = '041ba73b45d7d1948351b92073aef3fb77af348815ae9edbe6a29d98d5d7d3de1165dd7b1fb40ee534c0fba27def07cdfa64ae45522ddd4c4338a169f4606cac09'
len(raw)

130

In [47]:
cabecera = raw[0:2]
cabecera

'04'

In [48]:
raw2 = raw[2:]
mitad = len(raw2)//2

In [49]:
Qx = int(raw2[:mitad], 16)
Qy = int(raw2[mitad:], 16)

# Datos de la curva

Signature Algorithm: ecdsa_secp256r1_sha256

## Datos de la curva 256 del documento

In [50]:
p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
n = 115792089210356248762697446949407573529996955224135760342422259061068512044369
a = -3
b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b
Gx = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
Gy = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5

## Firma

In [51]:
raw = '3046022100914009da3d6ed684b32c63afe9c02eef32baa9ea6555ac50fad395bade9b5ae8022100cfeffb57304c724b15ab4fea764df1223f661846cb69df1e69f2fb7c9d992699'

### Extraer cabecera

In [52]:
print(raw[0:2])
raw = raw[2:]
print(raw)

30
46022100914009da3d6ed684b32c63afe9c02eef32baa9ea6555ac50fad395bade9b5ae8022100cfeffb57304c724b15ab4fea764df1223f661846cb69df1e69f2fb7c9d992699


### Comprobar longiutd de la cadena

In [53]:
longitud = raw[:2]
raw = raw[2:]
len(raw)//2 == int(longitud, 16)

True

### Obtener f1

In [54]:
print(raw[:2])
raw = raw[2:]

02


In [55]:
longitud = int(raw[:2], 16)
raw = raw[2:]
print(longitud)

33


In [56]:
f1 = raw[:longitud*2]
len(f1)

66

In [57]:
f1

'00914009da3d6ed684b32c63afe9c02eef32baa9ea6555ac50fad395bade9b5ae8'

In [58]:
raw = raw[longitud*2:]

### Obtener f2

In [59]:
print(raw[:2])
raw = raw[2:]

02


In [60]:
longitud = int(raw[:2], 16)
raw = raw[2:]
print(longitud)

33


In [61]:
f2 = raw[:longitud*2]
len(f2)

66

In [62]:
f2

'00cfeffb57304c724b15ab4fea764df1223f661846cb69df1e69f2fb7c9d992699'

In [63]:
raw = raw[longitud*2:]

### Raw deberia estar vacia

In [64]:
raw

''

# Mensaje firmado

## Preambulo

In [65]:
primero = '20' * 64
primero

'20202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020'

In [66]:
frase = 'TLS 1.3, server CertificateVerify'
segundo = ''
for letra in frase:
    numero = hex(ord(letra))[2:]
    numero = '0'*(2 - len(numero)) + numero
    segundo += numero
segundo

'544c5320312e332c20736572766572204365727469666963617465566572696679'

In [67]:
tercero = '00'

In [68]:
preambulo = primero + segundo + tercero
preambulo

'20202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020202020544c5320312e332c2073657276657220436572746966696361746556657269667900'

## Mensaje firmado

In [69]:
with open('mensaje.bin', 'rb') as f:
    mensaje = f.read()

In [70]:
from Crypto.Hash import SHA384
h = SHA384.new()
h.update(mensaje)
mensaje384 = h.hexdigest()

In [71]:
mensaje384

'34d37fd950b10f7fcc91eccf527459627d8c69b1ce6915ab5f7c460e1e7ce00359eb46c27274fe83f7fb07b2e8f583dc'

In [72]:
from Crypto.Hash import SHA256
h = SHA256.new()
h.update(bytes(bytearray.fromhex(preambulo + mensaje384)))
m = h.hexdigest()

In [73]:
m

'33243e6580a40631410a5fb8fae0ee67bdf36c447524ee295c2ae6a14252b19b'

# Verificacion firma

## Calculo pesos

In [74]:
w1 = (int(m, 16) * pow(int(f2, 16), -1, n)) % n
w1

87515200763454985111185401076111324364284015356633820489928573515422345095450

In [75]:
w2 = (int(f1, 16) * pow(int(f2, 16), -1, n)) % n
w2

89067318584195525323085404731611433077958851466746282182737511975670725719832

## Creacion curva y puntos

In [76]:
from Crypto.PublicKey.ECC import *
curva = 'p256'

In [77]:
G = EccPoint(Gx, Gy, curve=curva)
Q = EccPoint(Qx, Qy, curve=curva)

## Calculos en la curva, verificación firma

In [78]:
E = w1 * G + w2 * Q

In [79]:
x0, y0 = E.x, E.y

In [80]:
comprobacion = int(f1, 16) % n

In [81]:
comprobacion == x0

True

In [82]:
comprobacion

65698509256174583349959437356701870316724411153934220427135920273881720117992

In [83]:
int(x0)

65698509256174583349959437356701870316724411153934220427135920273881720117992

# Comprobad que el número de puntos (orden) de la curva usada en el certificado es primo.

In [84]:
from sympy import isprime
n = 115792089210356248762697446949407573529996955224135760342422259061068512044369
print('El orden de la curva es primo?', isprime(n))

El orden de la curva es primo? True


# Comprobad que la clave pública P de www.wikipedia.org es realmente un punto de la curva.

In [85]:
p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
izquierda = pow(Qy, 2, p)
izquierda

43799373942894144129896548519374924299177353174196618511093099420315891757984

In [86]:
derecha = (pow(Qx, 3, p) + a * Qx + b) % p
derecha

43799373942894144129896548519374924299177353174196618511093099420315891757984

In [87]:
print('Son iguales?', izquierda == derecha)

Son iguales? True


# Calculad el orden del punto P.

Al ser el orden de la curva un número primo, el orden del punto encaja.